Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Simple RAG with Vertex AI Search as a grounding service.

* This notebook explains how to use grounding service in Gemini Pro.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview
* Using Vertex AI Search :
  * https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview#ground-private

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [34]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [35]:
%pip install --upgrade --quiet google-cloud-discoveryengine

In [36]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [37]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.

In [38]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [39]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [40]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Vertex AI Search end point URL

In [41]:
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/servingConfigs/default_search:search"

In [42]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()

## Search contexts from Vertex AI Search

### REST API Call to retrieve contexts from Vertex AI Search

In [43]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{
            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text

### Parsing a document come from Vertex AI Search

In [44]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']
            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['content'] = chunk['content']

            # Chunks appear starting from those closest to the current Contents.
            p_chunks = chunk['chunkMetadata']['previousChunks']
            for p_chunk in p_chunks:
                item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

### Search context from Vertex AI Search

In [45]:
question = "침해사고의 원인 분석에 대해서 설명해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)

{
  "results": [
    {
      "chunk": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/branches/0/documents/3/chunks/c21",
        "id": "c21",
        "content": "\n# 제6장 정보통신망의 안정성 확보 등\n제48조의4(침해사고의 원인 분석 등) ① 정보통신서비스 제공자 등 정보통신망을 운영하는 자는 침 해사고가 발생하면 침해사고의 원인을 분석하고 그 결과에 따라 피해의 확산 방지를 위하여 사고 대응, 복구 및 재발 방지에 필요한 조치를 하여야 한다. \u003c개정 2022. 6. 10.\u003e ② 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 그 침해사고의 원인을 분석하고 피해 확산 방지, 사고대응, 복구 및 재발 방지를 위한 대책을 마련 하여 해당 정보통신서비스 제공자에게 필요한 조치를 하도록 권고할 수 있다.\u003c신설 2022. 6. 10.\u003e ③ 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 중대한 침해사고가 발생 한 경우 제2항에 따른 원인 분석 및 대책 마련을 위하여 필요하면 정보보호에 전문성을 갖춘 민 ·관합동조사단을 구성하여 그 침해사고의 원인 분석을 할 수 있다.\u003c개정 2013. 3. 23., 2017. 7. 26., 2022. 6. 10.\u003e ④ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 위하여 필요 하면 정보통신서비스 제공자에게 정보통신망의 접속기록 등 관련 자료의 보전을 명할 수 있다. \u003c개정 2013. 3. 23., 2017. 7. 26., 2022. 6. 10.\u003e ⑤ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 하기 

In [46]:
context = parse_chunks(searched_ctx)

print(context)

{'results-0': {'title': '정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123)', 'uri': 'gs://it_laws_kr/law_pdf/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf', 'content': '\n# 제6장 정보통신망의 안정성 확보 등\n⑦ 과학기술정보통신부장관은 인증에 관한 업무를 효율적으로 수행하기 위하여 필요한 경우 인 증심사 업무를 수행하는 기관(이하 "정보보호 관리체계 심사기관”이라 한다)을 지정할 수 있다. <신설 2015. 12. 1., 2017. 7. 26.> ⑧ 한국인터넷진흥원, 정보보호 관리체계 인증기관 및 정보보호 관리체계 심사기관은 정보보호 관리체계의 실효성 제고를 위하여 연 1회 이상 사후관리를 실시하고 그 결과를 과학기술정보통 신부장관에게 통보하여야 한다.<신설 2012. 2. 17., 2013. 3. 23., 2015. 12. 1., 2017. 7. 26.> ⑨ 제1항 및 제2항에 따라 정보보호 관리체계의 인증을 받은 자는 대통령령으로 정하는 바에 따 라 인증의 내용을 표시하거나 홍보할 수 있다.<개정 2012. 2. 17., 2015. 12. 1.> ⑩ 과학기술정보통신부장관은 다음 각 호의 어느 하나에 해당하는 사유를 발견한 경우에는 인증 을 취소할 수 있다. 다만, 제1호에 해당하는 경우에는 인증을 취소하여야 한다.<신설 2012. 2. 17., 2013. 3. 23., 2015. 12. 1., 2017. 7. 26.> 법제처 20 국가법령정보센터 정보통신망 이용촉진 및 정보보호 등에 관한 법률 1. 거짓이나 그 밖의 부정한 방법으로 정보보호 관리체계 인증을 받은 경우 2. 제4항에 따른 인증기준에 미달하게 된 경우 3. 제8항에 따른 사후관리를 거부 또는 방해한 경우

## Reasoning result with LLM

### Function to call LLM

In [47]:
def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

### Run example

In [48]:
from time import perf_counter

t1_start = perf_counter()

prompt = f"""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


Time : 10.18102411599989 seconds




정보통신망 이용촉진 및 정보보호 등에 관한 법률 제48조의4는 침해사고의 원인 분석에 대해 다음과 같이 규정하고 있습니다.

1. **정보통신서비스 제공자 등 정보통신망을 운영하는 자의 의무**: 침해사고가 발생하면 침해사고의 원인을 분석하고 그 결과에 따라 피해의 확산 방지를 위하여 사고 대응, 복구 및 재발 방지에 필요한 조치를 취해야 합니다. (제48조의4 제1항)

2. **과학기술정보통신부장관의 권한**: 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 그 침해사고의 원인을 분석하고 피해 확산 방지, 사고 대응, 복구 및 재발 방지를 위한 대책을 마련하여 해당 정보통신서비스 제공자에게 필요한 조치를 이행하도록 명령할 수 있습니다. (제48조의4 제2항)
   * 단, 공공기관등은 제외됩니다.

3. **과학기술정보통신부장관의 시정 명령**: 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 제2항에 따른 조치의 이행 여부를 점검하고, 보완이 필요한 사항에 대하여 해당 정보통신서비스 제공자에게 시정을 명할 수 있습니다. (제48조의4 제3항)

4. **민·관합동조사단 구성**: 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 중대한 침해사고가 발생한 경우 제2항에 따른 원인 분석 및 대책 마련을 위해 필요하다면 정보보호에 전문성을 갖춘 민·관합동조사단을 구성하여 침해사고의 원인 분석을 할 수 있습니다. (제48조의4 제4항)

5. **자료 보전 명령**: 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 위해 필요하다면 정보통신서비스 제공자에게 정보통신망의 접속기록 등 관련 자료의 보전을 명할 수 있습니다. (제48조의4 제5항)

6. **자료 제출 요구**: 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 위해 필요하다면 정보통신서비스 제공자에게 침해사고 관련 자료의 제출을 요구할 수 있으며, 중대한 침해사고의 경우 소속 공무원 또는 제4항에 따른 민·관합동조사단에게 관계인의 사업장에 출입하여 침해사고 원인을 조사하도록 할 수 있습니다. 다만, 「통신비밀보호법」제2조제11호에 따른 통신사실확인자료에 해당하는 자료의 제출은 같은 법으로 정하는 바에 따릅니다. (제48조의4 제6항)

7. **자료 보호 및 파기**: 과학기술정보통신부장관이나 민·관합동조사단은 제6항에 따라 제출받은 자료와 조사를 통해 알게 된 정보를 침해사고의 원인 분석 및 대책 마련 외의 목적으로는 사용하지 못하며, 원인 분석이 끝난 후에는 즉시 파기해야 합니다. (제48조의4 제7항)

8. **점검 방법, 조사 절차 등**: 제3항에 따른 점검의 방법·절차, 제4항에 따른 민·관합동조사단의 구성·운영, 제6항에 따라 제출된 자료의 보호 및 조사의 방법·절차 등에 필요한 사항은 대통령령으로 정합니다. (제48조의4 제8항)

결론적으로, 정보통신망 이용촉진 및 정보보호 등에 관한 법률은 정보통신망 운영자와 과학기술정보통신부장관에게 침해사고 발생 시 원인 분석을 의무화하고 있습니다. 과학기술정보통신부장관은 민·관합동조사단을 구성하여 침해사고를 조사할 수 있으며, 필요한 경우 정보통신서비스 제공자에게 관련 자료의 보전과 제출을 요구할 수 있습니다.  
